In [2]:
!python -m spacy download en_core_web_lg

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-29 00:00:06.219966: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-29 00:00:07.169342: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-29 00:00:07.169435: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open share

In [3]:
import os
import spacy
import string
from spacy.lang.en.stop_words import STOP_WORDS
from collections import Counter
import statistics
from statistics import mode

nlp = spacy.load("en_core_web_lg")

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [4]:
path = "/content/data"
dir_list = os.listdir(path)
os.chdir("/content/data")

universal_new_lines = '\n' + '\r' + '\r\n' + '\v' + '\x0b' + '\f' + '\x0c' + '\x1c' + '\x1d' + '\x1e' + '\x85' + '\u2028' + '\u2029' 
pun = string.punctuation + '\t' + '—' + '‘’' + universal_new_lines
processed_data = []

In [5]:
def data_processing(data):
  # removing punctuations and new lines and tabs
  for sym in pun:
    cnt = data.count(sym)
    data = data.replace(sym, '', cnt)

  # lower the data
  data = data.lower()
  doc = nlp(data)

  year = []
  gpe = []
  yr = ['2001', '2002', '2003', '2004']
  for token in doc:
    if token.pos_ == 'NUM' and token.shape_ == 'dddd' and token.text in yr:
      year.append(token.text)

  for ent in doc.ents:
    if ent.label_ == 'GPE':
      gpe.append(ent.text)

  if len(year) == 0:
    y = 'NA'
  else:
    y = mode(year)
  
  if len(gpe) == 0:
    g = 'NA'
  else:
    g = mode(gpe)

  return y, g

In [7]:
tokenization = {'2001': {}, '2002': {}, '2003': {}, '2004': {}, 'NA':{}}

for fname in dir_list:
  file = open(fname)
  c = 0
  content_list = file.readlines()
  while '\n' in content_list:
    content_list.remove('\n')
  for content in content_list:
    c = c + 1
    year, location = data_processing(content)
    filename = fname.replace("_docs.txt", str(c))
    if location in tokenization[year]:
      tokenization[year][location].append(filename)
    else:
      tokenization[year][location]=[]
      tokenization[year][location].append(filename)

print(tokenization)

{'2001': {'afghanistan': ['CIA33']}, '2002': {'detroit': ['FBI11'], 'toronto': ['CIA2'], 'khost area': ['CIA6'], 'cuba': ['CIA36']}, '2003': {'london': ['NSA1', 'FBI2'], 'amsterdam': ['NSA2', 'CIA13'], 'marrakech morocco': ['NSA4'], 'moscow': ['NSA5', 'CIA17'], 'nassau': ['NSA6'], 'santo domingo': ['NSA9', 'CIA3', 'CIA23'], 'paris': ['NSA10', 'CIA7'], 'montreal': ['FBI1'], 'omar evertts': ['FBI3'], 'chicago': ['FBI4', 'FBI6'], 'charlotte nc': ['FBI5'], 'toronto': ['FBI7'], 'virginia': ['FBI8'], 'texas': ['FBI9', 'FBI13', 'FBI19'], 'columbia': ['FBI10'], 'kansas': ['FBI12'], 'clifton nj': ['FBI14'], 'madrid': ['FBI16'], 'detroit': ['FBI23'], 'cairo': ['FBI24', 'CIA1', 'CIA24'], 'houston': ['FBI25'], 'calamar': ['CIA4'], 'peshawar': ['CIA8', 'CIA11', 'CIA15', 'CIA19'], 'casablanca': ['CIA12', 'CIA21'], 'windsor ontario': ['CIA16'], 'jeddah': ['CIA18'], 'NA': ['CIA20'], 'hyderabad': ['CIA22'], 'argentina': ['CIA30'], 'bahamas': ['CIA32']}, '2004': {'russia': ['NSA14'], 'tehran': ['NSA15']